In [4]:
%load_ext autoreload
%autoreload 2  

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
import torch

model = torch.hub.load("pytorch/vision", 'resnet50', weights="IMAGENET1K_V1")

Using cache found in /home/jupyter-steerapi/.cache/torch/hub/pytorch_vision_main


In [ ]:
import torch
import os
os.makedirs("_models", exist_ok=True)

modelpaths = [
    ("resnet50", "_models/resnet50.onnx"),
    ("alexnet", "_models/alexnet.onnx"),
    ("densenet121", "_models/densenet121.onnx"),
    ("mobilenet_v3_small", "_models/mobilenet_v3_small.onnx"),
    ("vgg16", "_models/vgg16.onnx"),
    ("vit_b_16", "_models/vit_b_16.onnx"),
]
for modelpath in modelpaths:
    if not os.path.exists(modelpath[1]):
        model = torch.hub.load("pytorch/vision", modelpath[0], weights="IMAGENET1K_V1")
        model.eval()
        torch.onnx.export(model, torch.ones(1,3,224,224), modelpath[1], verbose=True)

Using cache found in /home/jupyter-steerapi/.cache/torch/hub/pytorch_vision_main
/home/jupyter-steerapi/.conda/envs/stitchnet/lib/python3.10/site-packages/torch/__init__.py:833: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert condition, message
/home/jupyter-steerapi/.cache/torch/hub/pytorch_vision_main/torchvision/models/vision_transformer.py:273: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  n_h = h // p
/home/jupyter-steerapi/.cache/to

In [20]:
from torchvision.models.vision_transformer import ViT_B_16_Weights

transforms = ViT_B_16_Weights.IMAGENET1K_V1.transforms()

In [21]:
from PIL import Image
image = Image.open('_data/dogs_cats/train/cats/cat.1.jpg')

In [25]:
transforms(torch.ones(1,3,224,224)).shape

torch.Size([1, 3, 224, 224])

In [30]:
torch.onnx.export(model, torch.ones(1,3,224,224), modelpath[1], verbose=True)

In [31]:
model

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate=none)
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_att

In [27]:
# model
# modelpath[1]

In [5]:
import torch
import os

# evaluate accuracy of the original networks before finetuning
from onnx2torch import convert
from stitchnet.stitchonnx.utils import accuracy_score_model,accuracy_score_net,Net,get_score_net,load_cats_and_dogs_dset,load_dl,eval_original_model,get_macs_params_onnx,get_data_score

from stitchnet.stitchonnx.viz import draw_stitchNet_fromTuples
import torch
import os
from glob import glob
from tqdm import tqdm
from skl2onnx.helpers.onnx_helper import load_onnx_model
from pathlib import Path
data_score = get_data_score(32)

batch_size = 256
dataset_val = load_cats_and_dogs_dset("test")

modelpaths = [
    ("resnet50", "_models/resnet50.onnx"),
    ("alexnet", "_models/alexnet.onnx"),
    ("densenet121", "_models/densenet121.onnx"),
    ("mobilenet_v3_small", "_models/mobilenet_v3_small.onnx"),
    ("vgg16", "_models/vgg16.onnx"),
    ("vit_b_16", "_models/vit_b_16.onnx"),
]
# models = []
for modelpath in modelpaths:
    model = torch.hub.load("pytorch/vision", modelpath[0], weights="IMAGENET1K_V1")
    model.eval()
    
    acc = accuracy_score_model(model, dataset_val, batch_size)
    print(acc, modelpath[0])

Using cache found in /home/jupyter-steerapi/.cache/torch/hub/pytorch_vision_main
100%|██████████| 20/20 [00:06<00:00,  2.89it/s]
Using cache found in /home/jupyter-steerapi/.cache/torch/hub/pytorch_vision_main


0.8489194499017682 resnet50


100%|██████████| 20/20 [00:06<00:00,  3.02it/s]
Using cache found in /home/jupyter-steerapi/.cache/torch/hub/pytorch_vision_main


0.6998035363457761 alexnet


100%|██████████| 20/20 [00:07<00:00,  2.52it/s]
Using cache found in /home/jupyter-steerapi/.cache/torch/hub/pytorch_vision_main


0.8455795677799607 densenet121


100%|██████████| 20/20 [00:06<00:00,  2.97it/s]
Using cache found in /home/jupyter-steerapi/.cache/torch/hub/pytorch_vision_main


0.7797642436149312 mobilenet_v3_small


100%|██████████| 20/20 [00:10<00:00,  1.94it/s]


0.8117878192534381 vgg16
